### Multi-Label Classification

In [45]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import numpy as np

from copy import deepcopy 

In [46]:
train_rawdata = datasets.MNIST(root = 'dataset',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root = 'dataset',
                            train=False,
                            download=True,
                            transform=transforms.ToTensor())
print('number of training data : ', len(train_rawdata))
print('number of test data : ', len(test_dataset))

number of training data :  60000
number of test data :  10000


### Validation dataset 

In [47]:
VALIDATION_RATE = 0.2
train_indices, val_indices, _, _ = train_test_split(
    range(len(train_rawdata)), # X index 
    train_rawdata.targets, # y
    stratify=train_rawdata.targets, # 
    test_size=VALIDATION_RATE
)

### Create Validation dataset 

In [48]:
train_dataset = Subset(train_rawdata, train_indices)
validation_dataset = Subset(train_rawdata, val_indices)

### dataset size

In [49]:
print (len(train_dataset), len(validation_dataset), len(test_dataset))

48000 12000 10000


### Create Mini-batch 

In [50]:
minibatch_size = 128
# create batches
train_batches = DataLoader(train_dataset, batch_size=minibatch_size, shuffle=True)
val_batches = DataLoader(validation_dataset, batch_size=minibatch_size, shuffle=True)
test_batches = DataLoader(test_dataset, batch_size=minibatch_size, shuffle=False)

### Model

In [51]:
class Layer(nn.Module):
    def __init__(self, input_size, output_size, batch_norm=True, dropout=0.5):
        self.input_size = input_size
        self.output_size = output_size
        self.batch_norm = batch_norm
        self.dropout = dropout
        
        super().__init__()
        
        self.layer = nn.Sequential (
            nn.Linear(input_size, output_size),
            nn.LeakyReLU(), # LeakyReLU 기본값은 0.01
            self.apply_regularization()
        )
        
    def apply_regularization(self):
        if self.batch_norm:
            return nn.BatchNorm1d(self.output_size) # 입력 사이즈를 넣어줘야 함 (입력이 그 앞단의 Linear Layer 의 출력 사이즈가 됨)
        else:
            return nn.Dropout(self.dropout)
    
    def forward(self, x):
        return self.layer(x)